In [1]:
import pandas as pd

Variables à ajouter : 
- annee dans toutes les tables de matchs
- type_tournoi (elimatoire, round robine, elimatoire_qualif) dans toutes les tables de matchs


Création des fonctions

In [2]:
def ajout_annee(data):
    """
    Ajoute une colonne 'annee' au DataFrame correspond à la composante annee de "tourney_date".

    """
    # Conversion de int64 -> datetime
    data['tourney_date'] = pd.to_datetime(data['tourney_date'].astype(str), format="%Y%m%d", errors='coerce')

    # Création de la colonne année
    data['annee'] = data['tourney_date'].dt.year

    # Conversion en date simple (sans heure)
    data['tourney_date'] = data['tourney_date'].dt.date

    # Placer 'annee' en première position
    colonnes = ['annee'] + [col for col in data.columns if col != 'annee']
    data = data[colonnes]

    return data

In [7]:
def ajout_type_tournoi(data):
    """
    Ajoute une colonne 'type_tournoi' au DataFrame correspond à la composante type de "tourney_id".

    """
    # Initialisation de la colonne 'type_tournoi' à 'Eliminatoire'
    data['type_tournoi'] = 'Eliminatoire'

    # Récupération des types Round Robin -> tourney_id ou 'RR' est présent dans les rounds
    # Parcours par les id_tournois
    id_tournois = data['tourney_id'].unique()

    # initialisation
    data_result = pd.DataFrame()

    for id_tournoi in id_tournois:

        # Récupération des rounds du tournoi
        rounds = data[data['tourney_id'] == id_tournoi]['round'].unique()

        # Test Round Robin
        if 'RR' in rounds:
            data['type_tournoi'] = 'Round Robin'


        # Test Q1, Q2, Q3, Q4
        elif any(round_ in rounds for round_ in ['Q1', 'Q2', 'Q3', 'Q4']):
            data['type_tournoi'] = 'Qualification'

        data_result = pd.concat([data_result, data], axis=0)


    return data_result

In [8]:
def traitement():

    liste_fichier = ["../Donnees/atp_matches_1968_2024.csv",
                    "../Donnees/atp_matches_futures_1992_2024.csv",
                    "../Donnees/atp_matches_qual_1978_2024.csv"
                    "../Donnees/wta_matches_1968_2024.csv",
                    "../Donnees/wta_matches_qual_1968_2024.csv"
                    ]

    for fichier in liste_fichier:

        # Lecture du fichier
        data = pd.read_csv(fichier, index_col=False)

        # Ajout de la colonne 'annee'
        data = ajout_annee(data)

        # Ajout de la colonne 'type_tournoi'
        data = ajout_type_tournoi(data)

        # Enregistrement du fichier
        data.to_csv(fichier, index=False)

        print(f"Traitement du fichier {fichier} terminé.")


In [9]:
traitement()

KeyboardInterrupt: 